In [ ]:
#import all essential libraries
import sys 
import numpy as np # linear algebra
from scipy.stats import randint
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), data manipulation as in SQL
import matplotlib.pyplot as plt # this is used for the plot the graph 
import seaborn as sns # used for plot interactive graph. 
from sklearn.metrics import mean_squared_error,r2_score
## Deep-learing:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
df=pd.read_excel('Book01.xlsx')
df.head()

,Date,MB033,MB034,MB035,MB036
0,2021-01-01,site off,4,71,40
1,2021-01-02,88,66,52,12
2,2021-01-03,52,40,54,72
3,2021-01-04,49,34,32,28
4,2021-01-05,82,32,85,49


In [ ]:
df.tail()

,Date,MB033,MB034,MB035,MB036
493,2022-05-09,24,69,20,98
494,2022-05-10,1,19,74,6
495,2022-05-11,70,42,6,2
496,2022-05-12,92,3,99,64
497,2022-05-13,84,82,24,30


In [ ]:
#check for missing values in train data
df.isnull().sum()
 #No missing valuues



Date                     0
SKU MB-033 unit sold     0
SKU MB-034 unit sold     0
SKU MB-035 unit sold     0
SKU MB-036 unit sold     0
dtype: int64

Here for better understanding of the data, We can eloborate as month and weekday wise.

In [ ]:
df['MB034']

0      site off
1            88
2            52
3            49
4            82
         ...   
493          24
494           1
495          70
496          92
497          84
Name: MB033, Length: 498, dtype: object

In [ ]:
df['MB033'] = df['MB033'].replace(['site off'],0)
df['MB034'] = df['MB034'].replace(['site off'],0)
df['MB035'] = df['MB035'].replace(['site off'],0)
df['MB036'] = df['MB036'].replace(['site off'],0)









In [ ]:
arr1 = df['MB034'].values
arr1

array([  0,  88,  52,  49,  82,   0,  62,  77,  60,  37,  58,  64,  88,
        49,  38,  87,  22,  40,  85,  54,  87,  10,  60,  23,  66,  67,
        11,  76,  40,  68,  63,   0,  72,  67,  42,  85,  11,  22,  91,
        68,  19,  74,  45,   7,  57,   0,  54,  24,  80,  52,  51,  18,
         7,  72,  85,  77,  35,  93,  64,   0,  70,  97,   6,  42,  52,
        98,  11,  36,  82,  28,  84,   2,  68,  67,  54,   5,  82,  95,
        42,  72,  95,  94,  15,  42,  46,  95,  26,  91,  67,  31,   0,
        51,  30,  24,  72,  13, 100,  36,  69,  76,   5,  52,  71,  62,
        60,  57,  41,  39,  63,  63,  38,  93,   2,  39,  93,  44,  77,
        97,  54,  14,   0,  78,  27,  65,   9,   8,  36,  26,  54,  30,
        87,  39,  49,  20,  41,  33,  70,  89,   8,  76,  14,  63,  41,
        58,  61,  56,  20,  88,  58,  71,  55,   0,  88,  45,  39,  29,
        97,   1,  25,  23,  15,  47,  15,  47,  75,  21,  39,  30,  65,
        33,  94,   9,  70,  74,  72,  19,  38,  40,  71,  27,  1

In [ ]:
import numpy as np
def splitSequence(seq, n_steps):
    
    #Declare X and y as empty list
    X = []
    y = []
    
    for i in range(len(seq)):
        #get the last index
        lastIndex = i + n_steps
        
        #if lastIndex is greater than length of sequence then break
        if lastIndex > len(seq) - 1:
            break
            
        #Create input and output sequence
        seq_X, seq_y = seq[i:lastIndex], seq[lastIndex]
        
        #append seq_X, seq_y in X and y list
        X.append(seq_X)
        y.append(seq_y)
        pass
    #Convert X and y into numpy array
    X = np.array(X)
    y = np.array(y)
    
    return X,y 
    
    pass

n_steps = 20
X, y = splitSequence(arr1, n_steps = 20)

In [ ]:
X.shape,y.shape

((478, 20), (478,))

In [ ]:
for i in range(len(X)):
    print(X[i], y[i])

[ 0 88 52 49 82  0 62 77 60 37 58 64 88 49 38 87 22 40 85 54] 87
[88 52 49 82  0 62 77 60 37 58 64 88 49 38 87 22 40 85 54 87] 10
[52 49 82  0 62 77 60 37 58 64 88 49 38 87 22 40 85 54 87 10] 60
[49 82  0 62 77 60 37 58 64 88 49 38 87 22 40 85 54 87 10 60] 23
[82  0 62 77 60 37 58 64 88 49 38 87 22 40 85 54 87 10 60 23] 66
[ 0 62 77 60 37 58 64 88 49 38 87 22 40 85 54 87 10 60 23 66] 67
[62 77 60 37 58 64 88 49 38 87 22 40 85 54 87 10 60 23 66 67] 11
[77 60 37 58 64 88 49 38 87 22 40 85 54 87 10 60 23 66 67 11] 76
[60 37 58 64 88 49 38 87 22 40 85 54 87 10 60 23 66 67 11 76] 40
[37 58 64 88 49 38 87 22 40 85 54 87 10 60 23 66 67 11 76 40] 68
[58 64 88 49 38 87 22 40 85 54 87 10 60 23 66 67 11 76 40 68] 63
[64 88 49 38 87 22 40 85 54 87 10 60 23 66 67 11 76 40 68 63] 0
[88 49 38 87 22 40 85 54 87 10 60 23 66 67 11 76 40 68 63  0] 72
[49 38 87 22 40 85 54 87 10 60 23 66 67 11 76 40 68 63  0 72] 67
[38 87 22 40 85 54 87 10 60 23 66 67 11 76 40 68 63  0 72 67] 42
[87 22 40 85 54 87 10 60 2

In [ ]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
print(X[:2])

[[[ 0]
  [88]
  [52]
  [49]
  [82]
  [ 0]
  [62]
  [77]
  [60]
  [37]
  [58]
  [64]
  [88]
  [49]
  [38]
  [87]
  [22]
  [40]
  [85]
  [54]]

 [[88]
  [52]
  [49]
  [82]
  [ 0]
  [62]
  [77]
  [60]
  [37]
  [58]
  [64]
  [88]
  [49]
  [38]
  [87]
  [22]
  [40]
  [85]
  [54]
  [87]]]


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model = tf.keras.Sequential()
model.add(layers.LSTM(100, activation='relu', input_shape=(n_steps, n_features)))
model.add(layers.Dense(1))

In [ ]:
model.layers

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 100)               40800     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=['accuracy'])

In [ ]:
model.fit(X, y, epochs=200, verbose=1)

Epoch 1/200
15/15 [==============================] - 3s 15ms/step - loss: 123951.7188 - accuracy: 0.0063
Epoch 2/200
15/15 [==============================] - 0s 32ms/step - loss: 2480.3467 - accuracy: 0.0063
Epoch 3/200
15/15 [==============================] - 0s 15ms/step - loss: 1358.8363 - accuracy: 0.0063
Epoch 4/200
15/15 [==============================] - 0s 32ms/step - loss: 929.3257 - accuracy: 0.0063
Epoch 5/200
15/15 [==============================] - 0s 27ms/step - loss: 893.3033 - accuracy: 0.0063
Epoch 6/200
15/15 [==============================] - 0s 23ms/step - loss: 888.3137 - accuracy: 0.0063
Epoch 7/200
15/15 [==============================] - 0s 30ms/step - loss: 907.8947 - accuracy: 0.0063
Epoch 8/200
15/15 [==============================] - 0s 33ms/step - loss: 897.4164 - accuracy: 0.0063
Epoch 9/200
15/15 [==============================] - 0s 30ms/step - loss: 857.9556 - accuracy: 0.0063
Epoch 10/200
15/15 [==============================] - 0s 29ms/step - loss: 85

In [ ]:
test_data = arr1[-101:-1]
test_data

array([56, 17,  8, 63, 86, 67, 40,  7, 38, 25,  3, 58, 92, 26, 98, 15, 44,
       24, 77, 29, 42, 43, 82, 44, 59, 85, 86,  0, 92, 66, 59, 30, 69, 53,
       50, 64, 12, 95, 47, 38, 94, 84, 40, 10, 62, 98, 89, 31, 59, 68, 95,
       97, 91, 89, 57, 87, 34, 87,  0, 35, 25, 66, 53, 21, 12, 77, 31, 71,
       49, 99,  7, 15, 99, 86, 69, 92, 61, 52, 68, 83, 67, 13, 77, 15, 68,
       48, 80, 91,  0, 35, 28, 97, 25, 13, 24, 22, 24,  1, 70, 92])

In [ ]:
test_data = arr1[-20:]
test_data = test_data.reshape((1, n_steps, n_features))
test_data

array([[[13],
        [77],
        [15],
        [68],
        [48],
        [80],
        [91],
        [ 0],
        [35],
        [28],
        [97],
        [25],
        [13],
        [24],
        [22],
        [24],
        [ 1],
        [70],
        [92],
        [84]]])

In [ ]:
test_data.mean()

45.35

In [ ]:
predictNextNumber = model.predict(test_data, verbose=1)
print(predictNextNumber)

1/1 [==============================] - 0s 191ms/step
[[75.57227]]
